In [6]:
import torch
def expand(st_dict, new_path):
    st_dict['layoutlmv3.embeddings.position_ids'] = torch.cat(
        (st_dict['layoutlmv3.embeddings.position_ids'], st_dict['layoutlmv3.embeddings.position_ids'])
        , dim=1)
    st_dict['layoutlmv3.embeddings.position_embeddings.weight'] = torch.cat(
        (st_dict['layoutlmv3.embeddings.position_embeddings.weight'], st_dict['layoutlmv3.embeddings.position_embeddings.weight'])
        , dim=0)
    print("expand position_ids to ", st_dict['layoutlmv3.embeddings.position_ids'].shape)
    print("expand position_embeddings.weight to ", st_dict['layoutlmv3.embeddings.position_embeddings.weight'].shape)
    torch.save(st_dict, new_path)

def expand_truncate(st_dict, new_path):
    st_dict['layoutlmv3.embeddings.position_ids'] = torch.cat(
        (st_dict['layoutlmv3.embeddings.position_ids'], st_dict['layoutlmv3.embeddings.position_ids'])
        , dim=1)
    st_dict['layoutlmv3.embeddings.position_embeddings.weight'] = torch.cat(
        (st_dict['layoutlmv3.embeddings.position_embeddings.weight'], st_dict['layoutlmv3.embeddings.position_embeddings.weight'])
        , dim=0)
    print("expand position_ids to ", st_dict['layoutlmv3.embeddings.position_ids'].shape)
    print("expand position_embeddings.weight to ", st_dict['layoutlmv3.embeddings.position_embeddings.weight'].shape)
    for k in list(st_dict.keys()):
        if k.startswith('classifier.'):
            st_dict.pop(k)
            print("pop ", k)
    torch.save(st_dict, new_path)

In [1]:
from layoutlmft.models.layoutlmv3.modeling_layoutlmv3 import LayoutLMv3ForRelationExtraction, LayoutLMv3ForTokenClassification
model_ft = LayoutLMv3ForTokenClassification.from_pretrained(
    "HYPJUDY/layoutlmv3-large-finetuned-funsd"
)

/root/mambaforge/envs/env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of LayoutLMv3ForTokenClassification were not initialized from the model checkpoint at HYPJUDY/layoutlmv3-large-finetuned-funsd and are newly initialized: ['layoutlmv3.encoder.layer.16.attention.self.lam', 'layoutlmv3.encoder.layer.5.attention.self.lam', 'layoutlmv3.encoder.layer.19.attention.self.lam', 'layoutlmv3.encoder.layer.8.attention.self.lam', 'layoutlmv3.encoder.layer.7.attention.self.lam', 'layoutlmv3.encoder.layer.21.attention.self.lam', 'layoutlmv3.encoder.layer.15.attention.self.lam', 'layoutlmv3.encoder.layer.10.attention.self.lam', 'layoutlmv3.encoder.layer.9.attention.self.lam', 'layoutlmv3.encoder.layer.17.attention.self.lam', 'layoutlmv3.encoder.layer.6.attention.self.lam', 'layoutlmv3.encoder.layer.14

In [2]:
re_pretrain = LayoutLMv3ForRelationExtraction.from_pretrained(
    "microsoft/layoutlmv3-large"
)
ner_pretrain = LayoutLMv3ForTokenClassification.from_pretrained(
    "microsoft/layoutlmv3-large"
)

Some weights of LayoutLMv3ForRelationExtraction were not initialized from the model checkpoint at microsoft/layoutlmv3-large and are newly initialized: ['extractor.ffnn_head.3.bias', 'layoutlmv3.encoder.layer.16.attention.self.lam', 'layoutlmv3.encoder.layer.5.attention.self.lam', 'extractor.ffnn_tail.3.weight', 'layoutlmv3.encoder.layer.19.attention.self.lam', 'layoutlmv3.encoder.layer.8.attention.self.lam', 'layoutlmv3.encoder.layer.7.attention.self.lam', 'layoutlmv3.encoder.layer.21.attention.self.lam', 'extractor.ffnn_head.3.weight', 'layoutlmv3.encoder.layer.15.attention.self.lam', 'layoutlmv3.encoder.layer.10.attention.self.lam', 'extractor.rel_classifier.linear.weight', 'extractor.rel_classifier.bilinear.weight', 'extractor.rel_classifier.linear.bias', 'extractor.ffnn_tail.3.bias', 'extractor.ffnn_head.0.bias', 'layoutlmv3.encoder.layer.9.attention.self.lam', 'layoutlmv3.encoder.layer.17.attention.self.lam', 'layoutlmv3.encoder.layer.6.attention.self.lam', 'layoutlmv3.encoder.la

In [22]:
re_st_dict = re_pretrain.state_dict()
expand(re_st_dict, "layoutlmv3-large-re-1028/pytorch_model.bin")

expand position_ids to  torch.Size([1, 1028])
expand position_embeddings.weight to  torch.Size([1028, 1024])


OrderedDict([('layoutlmv3.cls_token',
              tensor([[[ 0.0050,  0.0054, -0.0062,  ...,  0.0047,  0.0048, -0.0034]]])),
             ('layoutlmv3.pos_embed',
              tensor([[[ 0.0050,  0.0054, -0.0062,  ...,  0.0047,  0.0048, -0.0034],
                       [-0.0228, -0.1192, -0.0385,  ...,  0.0437,  0.0281,  0.0316],
                       [-0.0021, -0.0172, -0.0013,  ...,  0.0197,  0.0136, -0.0174],
                       ...,
                       [-0.0248,  0.0350, -0.0216,  ...,  0.0213,  0.0110,  0.0308],
                       [-0.0320,  0.0189, -0.0041,  ...,  0.0219, -0.0354,  0.0519],
                       [-0.0391, -0.0269,  0.0372,  ..., -0.0059, -0.0987,  0.0855]]])),
             ('layoutlmv3.embeddings.position_ids',
              tensor([[  0,   1,   2,  ..., 511, 512, 513]])),
             ('layoutlmv3.embeddings.word_embeddings.weight',
              tensor([[-0.1048,  0.0187,  0.0803,  ...,  0.0456,  0.0124, -0.0310],
                      [ 0.0069, 

In [7]:
ner_st_dict = ner_pretrain.state_dict()
expand_truncate(ner_st_dict, "layoutlmv3-large-ner-1028/pytorch_model.bin")

expand position_ids to  torch.Size([1, 1028])
expand position_embeddings.weight to  torch.Size([1028, 1024])
pop  classifier.weight
pop  classifier.bias


In [4]:
for k,v in model_ft.named_parameters():
    print(k, v.size())

layoutlmv3.cls_token torch.Size([1, 1, 1024])
layoutlmv3.pos_embed torch.Size([1, 197, 1024])
layoutlmv3.embeddings.word_embeddings.weight torch.Size([50265, 1024])
layoutlmv3.embeddings.token_type_embeddings.weight torch.Size([1, 1024])
layoutlmv3.embeddings.LayerNorm.weight torch.Size([1024])
layoutlmv3.embeddings.LayerNorm.bias torch.Size([1024])
layoutlmv3.embeddings.position_embeddings.weight torch.Size([514, 1024])
layoutlmv3.embeddings.x_position_embeddings.weight torch.Size([1024, 171])
layoutlmv3.embeddings.y_position_embeddings.weight torch.Size([1024, 171])
layoutlmv3.embeddings.h_position_embeddings.weight torch.Size([1024, 170])
layoutlmv3.embeddings.w_position_embeddings.weight torch.Size([1024, 170])
layoutlmv3.encoder.layer.0.attention.self.lam torch.Size([])
layoutlmv3.encoder.layer.0.attention.self.query.weight torch.Size([1024, 1024])
layoutlmv3.encoder.layer.0.attention.self.query.bias torch.Size([1024])
layoutlmv3.encoder.layer.0.attention.self.key.weight torch.Siz

In [17]:
%env http_proxy=127.0.0.1:11070
%env https_proxy=127.0.0.1:11070

env: http_proxy=127.0.0.1:11070
env: https_proxy=127.0.0.1:11070
